In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
!ls "drive/MyDrive"

 201102_AADHAAR_CARD.pdf
 201102_Photo.jpeg
 201102_X_MARKSHEET.pdf
 ACM_EXECS.gsheet
 ACM_ML_6.pdf
'ACM REC 23.gsheet'
'antiragging_form (1).pdf'
 antiragging_form.pdf
 CNN_Plant_Data
'Colab Notebooks'
'Committee Application (1).docx'
'Committee Application.docx'
'Complex Systems.gsheet'
'Development 1 and 2 combined.dwg'
'Development 1 and 2 combined-Model.pdf'
'dru-wyxj-kcb – 9 Sept 2023.gjam'
'Eshan Joshi A020 Btech IT Sem 3 certificate.pdf'
 EshanJoshi_ACM_TECH_SOP.docx
 EshanJoshi_Semicode_A1.docx
 EshanJoshi_Semicode_A3.docx
 EshanJoshi_Semicode_A4.docx
'Eshan Joshi_Semicode_A5.docx'
 EshanJoshi_Semicode_A6.docx
 EshanJoshi_SemiCode_Q1.docx
'EshanJoshi_Semicode_Q2 (1).docx'
'EshanJoshi_Semicode_Q2 (2).docx'
 EshanJoshi_Semicode_Q2.docx
 EshanJoshi_timetable.pdf
 Eshan_Tech.png
 IMG_1116.png
 Minecraft
'OnePlus review '
'PFE Debuggers.gform'
'PFE Debuggers (Responses).gsheet'
 reviewbyEshan.jpg
 Screenshot_20220105-105158.jpg
 Screenshot_2022-12-02-22-14-52-68_1c337646f29875672b5

NameError: name 'clear_output' is not defined

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import os
from PIL import Image
from matplotlib.image import imread
from os import listdir
from sklearn.preprocessing import label_binarize, LabelBinarizer
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array, array_to_img
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, Activation
from sklearn.model_selection import train_test_split
from keras.models import model_from_json
from keras.utils import to_categorical

In [15]:
def convert_img_to_array(img_dir):
  try:
    image = cv2.imread(img_dir)
    if image is not None:
      image = cv2.resize(image, (256, 256))
      return image
    else:
      return np.array([])
  except Exception as e:
    print(e)
    return None

In [17]:
dir = "drive/MyDrive/CNN_Plant_Data"
root_dir = listdir(dir)
image_list, label_list = [],[]
all_labels = ['Corn-Common_rust','Potato-Early_blight','Tomato-Bacterial_spot']
binary_labels = [0,1,2]
temp = -1



for dir1 in root_dir:
  plant_image_list = listdir(f"{dir}/{dir1}")
  temp = temp+1
  for image in plant_image_list:
    image_path = f"{dir}/{dir1}/{image}"
    image_list.append(convert_img_to_array(image_path))
    label_list.append(binary_labels[temp])

In [19]:
label_counts = pd.DataFrame(label_list).value_counts()
label_counts

0    300
1    300
2    300
Name: count, dtype: int64

In [20]:
x_train, x_test, y_train, y_test = train_test_split(image_list, label_list, test_size=0.2, random_state=10)

In [21]:
x_train = np.array(x_train, dtype=np.float16) / 225.0
x_test = np.array(x_test, dtype=np.float16) / 225.0
x_train = x_train.reshape( -1, 256,256,3)
x_test = x_test.reshape( -1, 256,256,3)

In [22]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [24]:
model = Sequential()
model.add(Conv2D(32,(3,3),padding="same",input_shape=(256,256,3),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),padding="same",activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dense(3,activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 64, 64, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 262144)            0         
                                                                 
 dense (Dense)               (None, 64)                1

In [25]:
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.0001),metrics=['accuracy'])

In [26]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)

In [27]:
epochs = 100
batch_size = 128
history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs,
                    validation_data = (x_val, y_val))

Epoch 1/100
5/5 [==============================] - 16s 1s/step - loss: 1.1901 - accuracy: 0.4757 - val_loss: 0.4387 - val_accuracy: 0.7292
Epoch 2/100
5/5 [==============================] - 1s 205ms/step - loss: 0.4860 - accuracy: 0.7431 - val_loss: 0.2063 - val_accuracy: 0.9306
Epoch 3/100
5/5 [==============================] - 1s 200ms/step - loss: 0.2707 - accuracy: 0.9167 - val_loss: 0.2903 - val_accuracy: 0.8681
Epoch 4/100
5/5 [==============================] - 1s 208ms/step - loss: 0.2387 - accuracy: 0.9306 - val_loss: 0.1380 - val_accuracy: 0.9583
Epoch 5/100
5/5 [==============================] - 1s 207ms/step - loss: 0.1516 - accuracy: 0.9618 - val_loss: 0.0926 - val_accuracy: 0.9861
Epoch 6/100
5/5 [==============================] - 1s 205ms/step - loss: 0.1075 - accuracy: 0.9688 - val_loss: 0.0745 - val_accuracy: 0.9931
Epoch 7/100
5/5 [==============================] - 1s 203ms/step - loss: 0.0911 - accuracy: 0.9809 - val_loss: 0.0953 - val_accuracy: 0.9653
Epoch 8/100
5/5